In [1]:
from datetime import date, timedelta
import feedparser
import json
import time
import pandas as pd

In [2]:

def google_news_data(query,start_date,end_date, skip_range,sleep): 
    """
    Google news RSS for scraping news by query and time window.
  
    RSS details: https://newscatcherapi.com/blog/google-news-rss-search-parameters-the-missing-documentaiton
  
    Parameters:
    query (str): Keywords divided by OR 
    start_date (date): Time window start date
    end_date (date): Time window end date
    skip_range (int): Aggregating news over a number of days depending on the use case(More aggregation means less time to collect data)
    sleep (int): Google news API is throttled. Avoid it by setting a sleep time between calls
  
    Returns:
    dict: End date as the key and the array of news for that window as an array
  
    """
    delta = end_date - start_date   # returns timedelta
    final_json = {}
    range_counter = 1
    # print(delta.days)
    for i in range(delta.days + 1):
        if range_counter == skip_range:
            session_start_date = start_date + timedelta(days=i-skip_range) 
            session_end_date = start_date + timedelta(days=i) 
            print(session_end_date)
            query = query.replace(" ", "%20")  
            query = query.replace("&", "and")  
            NewsFeed = feedparser.parse("https://news.google.com/rss/search?q="+query+"+before:"+session_end_date.strftime("%Y-%m-%d")+"+after:"+session_start_date.strftime("%Y-%m-%d")+"&ceid=US:en&hl=en-US&gl=US")
            time.sleep(sleep)
            final_json[session_end_date.strftime("%Y-%m-%d")] = NewsFeed.entries
            range_counter = 1 
        else:
            range_counter += 1
            continue
    out_file = open('News_'+query.replace(" ","_").replace("%20"," ")+".json", "w") 
    json.dump(final_json, out_file, indent = 6)         
    out_file.close() 
    return final_json


In [ ]:
# skip_range is to aggregate news scraping by several days, sleep is the time to wait to call the api to avoid throttling
output = google_news_data(query = "USD OR EUR OR YEN",start_date = date(2021, 8, 1),end_date = date(2022, 1, 1), skip_range=1,sleep=60)

2021-08-01
2021-08-02
2021-08-03


In [ ]:
output

In [ ]:
f = open('News_USD OR EUR OR YEN.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)
  

In [ ]:
data['2022-01-01']

In [ ]:
collate=[]
for i in data.keys():
    for item in data[i]:
        title=pd.Series(item['title'])
        date=pd.Series(item['published'])
        frame = { 'title': title, 'date': date }
#Creating DataFrame by passing Dictionary
        df = pd.DataFrame(frame)
        collate.append(df)

In [ ]:
pd.concat(collate) 